In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [4]:
ds = tf.data.TFRecordDataset('/vision/data/invoices_train.tfrecord')

In [5]:
for element in ds.take(1):
    print(element)

tf.Tensor(b'\n\x82\xd9-\n\xee\xd8-\n\timage_raw\x12\xdf\xd8-\n\xdb\xd8-\n\xd7\xd8-\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00\xfa\x00\xfa\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\n\xbe\x08M\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\x

In [5]:
def parse_tfrecords(tensor_string):
    """ 
    Decode a serialized tensor
    That takes a image_dicts normalize and returns a tuple (x,y)
    Returns a x(image) to decode
    """
    feature_decode = {
        'image_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'label': tf.io.FixedLenFeature([], tf.int64, default_value=0)}

    img_dic = tf.io.parse_single_example(tensor_string, feature_decode)
    
    label = img_dic['label']
    image = tf.io.parse_tensor(serialized=img_dic["image_raw"], out_type=tf.uint8)

    return {'image': image, 'label': label}

In [6]:
def parse_tfrecords(tensor_string):
    """ 
    Decode a serialized tensor
    That takes a image_dicts normalize and returns a tuple (x,y)
    Returns a x(image) to decode
    """
    feature_decode = {
        'image_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'label': tf.io.FixedLenFeature([], tf.int64, default_value=0)}

    img_dic = tf.io.parse_single_example(tensor_string, feature_decode)
    
    label= img_dic['label']
    image = tf.image.decode_jpeg(img_dic['image_raw'])

    return {'image': image, 'label': label}

In [7]:
ds1 = ds.map(parse_tfrecords)
ds1

<MapDataset shapes: {image: (None, None, None), label: ()}, types: {image: tf.uint8, label: tf.int64}>

In [9]:
for element in ds1.take(1):
    print(tf.image.resize(element['image'], [224, 224]))

tf.Tensor(
[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]], shape=(224, 224, 3), dtype=float32)


In [11]:
def preprocess_data(image_dic):
    """  
    Function that takes a image_dic, normalized (divided by 255) and returns a tupe (x, y)
    """

    image, label = image_dic['image'], image_dic['label']
    
    image.set_shape([None, None, None])
    #image = tf.image.decode_jpeg(image)
    image = tf.math.divide(image, 255)
    image = tf.image.resize(image, (224, 224))
    image = tf.reshape(image, [224*224*3])

    return (image, label)

ds2 = ds1.map(preprocess_data)
ds2

<MapDataset shapes: ((150528,), ()), types: (tf.float32, tf.int64)>

In [12]:
for element in ds2.take(1):
    print(element)

(<tf.Tensor: shape=(150528,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [13]:
inputs = tf.keras.Input(shape=(224*224*3,), name='Hiraganas')
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.BatchNormalization(renorm=True)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.BatchNormalization(renorm=True)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
# "sparse me sirve para clasificacion mutuamente excluyente un label --- multilabel"
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [14]:
ds2 = ds2.shuffle(buffer_size=30000).batch(32)
ds2 = ds2.batch(64)
history = model.fit(ds2, epochs=3)

ValueError: in converted code:

    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training_v2.py:677 map_fn
        batch_size=None)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training.py:2410 _standardize_tensors
        exception_prefix='input')
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training_utils.py:573 standardize_input_data
        'with shape ' + str(data_shape))

    ValueError: Error when checking input: expected Hiraganas to have 2 dimensions, but got array with shape (None, None, 150528)
